In [77]:
import pandas as pd
import time
import numpy as np
np.seterr(invalid='ignore')
import nbimporter
from PPP import PPP
import os

game = 'Game5'
Opp = 'Nuggets'
Team = 'Heat'
series = 'Finals'
add = True

In [72]:
players = ['adebayo', 'butler', 'highsmith', 'love', 'lowry', 'martin', 'robinson', 'strus', 'vincent', 'zeller', 'team']
opp = ['opp']

In [78]:
def load_data(players):
    
    # Establish dictionaries for new game data and overall series/season data
    new_player_data = {}
    ovr_player_data = {}
    ovr_series_player_data = {}
    team_opp = ['team', 'opp']

    for player in players: # Loop over all players on roster

        if player == 'team': # Add all player data together to create team file for game

            try: # to load a team file for the game (helpful if already added team file, and/or rerunning for Opp file)
                newplayerFile = (f'Team/{game + Opp}Team.csv')
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.dropna()
                newplayer = newplayer.set_index('Shot Type')
                newplayer = newplayer.astype('int')
                new_player_data[player] = newplayer # Add the existing new team file to the new_player_data dictionary
            except(FileNotFoundError):
                # Load an empty df
                newplayerFile = ('Team/empty.csv')
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.set_index('Shot Type')

                for key, df in new_player_data.items(): # Loop over the new_player_data and add together all player dataframes
                    if key not in team_opp: # Exclude the team and opponent dataframes
                        newplayer = newplayer.add(df)

                new_player_data['team'] = newplayer # Add the created new team file to new_player_data dictionary
                newplayer.to_csv(f'../{Opp}/Opp/{game + Team}Opp.csv') # Save team file to opponent folder for their opponent stats
                newplayer.to_csv(f'Team/{game + Opp}Team.csv') # Save team file to team folder

            # Load overall data for the team
            try:
                ovrplayerFile = (f'!Data/Team.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add the overall team data to the ovr_player_data dictionary
            except(FileNotFoundError):
                print(f'No overall team file found.')
                ovrplayer = newplayer
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv')

            # Load overall series data for the player
            try: 
                ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
                ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                ovr_seriesplayer = ovr_seriesplayer.dropna()
                ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                ovr_seriesplayer = ovr_seriesplayer.astype('int')
                ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
            except(FileNotFoundError):
                ovr_seriesplayer = newplayer
                ovr_series_player_data[player] = ovr_seriesplayer
                ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
                continue
                
            if add == True: # Check to see if the game is supposed to be added to the overall and series overall data
                if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory

                    ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
                else:
                    print(f"{player.title()}'s columns do not match")  
            continue

        if player == 'opp': # Load the opponent file and add the new game data to the overall opponent file if necessary
            
            # Load the new opponent file (if running this teams data first, there will be no opponent file!!)
            newplayerFile = (f'Opp/{game + Opp}Opp.csv')
            if os.path.exists(newplayerFile):
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.dropna()
                newplayer = newplayer.set_index('Shot Type')
                newplayer = newplayer.astype('int')
                new_player_data[player] = newplayer # Add the new opponent file to the new_player_data dictionary
            else:
                print(f'No {player.title()} file found.')
                continue

            # Load overall data for the opponent
            ovrplayerFile = '!Data/Opp.csv'
            if os.path.exists(ovrplayerFile):
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add the overall opponent file to the ovr_player_data dictionary
            elif os.path.exists(newplayerFile):
                ovrplayer = newplayer
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv')
            else:
                print(f'No overall Opponent file found.')
                continue

            # Load overall series data for the player
            try: 
                ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
                ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                ovr_seriesplayer = ovr_seriesplayer.dropna()
                ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                ovr_seriesplayer = ovr_seriesplayer.astype('int')
                ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
            except(FileNotFoundError):
                ovr_seriesplayer = newplayer
                ovr_series_player_data[player] = ovr_seriesplayer
                ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
                continue
            
            if add == True:
                if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                    if game == 'Game1':
                        continue

                    ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
                else:
                    print(f"{player.title()}'s columns do not match")  
            continue

        ### Loading/Adding the player files (first part of the loop)###
        try:
            newplayerFile = (f'{player.title()}/{game + Opp + player.title()}.csv') # Search the team directory for the player, then concat the game/opp/player to find csv for game
            newplayer = pd.read_csv(newplayerFile)
            newplayer = newplayer.dropna()
            newplayer = newplayer.set_index('Shot Type')
            newplayer = newplayer.astype('int')
            new_player_data[player] = newplayer # Add the new player data to the new_player_data dictionary
        except(FileNotFoundError):
            print(f'No {player.title()} file found for {game}.')
            try: # to load an overall file for the player
                ovrplayerFile = (f'!Data/{player.title()}.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
                continue
            except(FileNotFoundError):
                print(f'No data for {player.title()}.')
                continue # If no player data is found, skip and go to the next player (edit for later)

            try: # to load an overall player file for the series
                ovr_seriesplayerFile = (f'!Data/{player.title()}.csv')
                ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                ovr_seriesplayer = ovr_seriesplayer.dropna()
                ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                ovr_seriesplayer = ovr_seriesplayer.astype('int')
                ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_series_player_data dictionary
                continue
            except(FileNotFoundError):
                print(f'No {series} series data for {player.title()}.')
                continue # If no player data is found, skip and go to the next player (edit for later)

        # Load overall data for the player
        try: 
            ovrplayerFile = (f'!Data/{player.title()}.csv')
            ovrplayer = pd.read_csv(ovrplayerFile)
            ovrplayer = ovrplayer.dropna()
            ovrplayer = ovrplayer.set_index('Shot Type')
            ovrplayer = ovrplayer.astype('int')
            ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
        # If there is no overall data for the player, set the new player data as the overall and continue to the next player    
        except(FileNotFoundError):
            ovrplayer = newplayer
            ovr_player_data[player] = ovrplayer
            ovrplayer.to_csv(f'!Data/{player.title()}.csv')

        # Load overall series data for the player
        try: 
            ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
            ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
            ovr_seriesplayer = ovr_seriesplayer.dropna()
            ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
            ovr_seriesplayer = ovr_seriesplayer.astype('int')
            ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
        # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
        except(FileNotFoundError):
            ovr_seriesplayer = newplayer
            ovr_series_player_data[player] = ovr_seriesplayer
            ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
            continue
        
        if add == True:
            if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                ovrplayer = ovrplayer.add(newplayer)
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                if game == 'Game1':
                    continue

                ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                ovr_series_player_data[player] = ovr_seriesplayer
                ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
            else:
                print(f"{player.title()}'s columns do not match")  
        continue
    print('Complete.')
    return new_player_data, ovr_player_data, ovr_series_player_data # Return both dictionaries as a tuple

new_player_data, ovr_player_data, ovr_series_player_data  = load_data(opp) # Unpack the tuple of dictionaries into their respective variables

Complete.


In [28]:
newButlerStats = PPP(new_player_data['butler'])
newButlerStats.to_csv(f'Butler/PPP Stats/{game + Opp}.csv')
newButlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.27,35.5,0,9.1,0.38,36.4,12.5,6,0.0,2,50.0,0.0,33.3,0.0,1,0.0,2,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.52,4.3,0,N/A,1.52,6.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.0,6.5,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.0,22.2,50.0,1,100.0,1,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.74,35.1,1,33.3,0.76,35.8,28.6,6,33.3,1,0.0,0.67,33.3,50.0,1,100.0,1,0.0
TRANSITION,2.13,6.1,0,100.0,2.13,8.5,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.06,6.1,0,0.0,2.27,4.0,N/A,0,N/A,0,N/A,0.0,11.1,0.0,1,0.0,0,N/A
Catch & Shoot,1.5,6.5,0,50.0,1.5,9.1,50.0,0,N/A,2,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [29]:
ButlerSeriesStats = PPP(ovr_series_player_data['butler'])
ButlerSeriesStats.to_csv(f'Butler/PPP Stats/{series}/{Opp + series}.csv')
ButlerSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.8,33.8,2,34.0,0.53,31.4,25.0,26,23.1,6,33.3,1.26,39.0,50.0,9,44.4,9,55.6
PNR Screener,0.0,0.6,0,0.0,0.0,0.9,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.86,1.5,0,0.0,0.86,2.1,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.04,1.8,0,50.0,1.14,0.8,N/A,0,N/A,0,N/A,1.0,3.9,50.0,1,100.0,1,0.0
DBL Ball Handler,1.33,1.9,0,66.7,2.0,1.9,100.0,2,100.0,0,N/A,0.0,2.0,0.0,0,N/A,1,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.13,35.1,2,46.9,1.1,30.4,44.4,22,50.0,5,20.0,1.17,45.1,50.0,8,75.0,14,35.7
TRANSITION,1.14,6.1,0,42.9,1.27,8.0,50.0,6,50.0,0,N/A,0.0,2.0,0.0,1,0.0,0,N/A
Attacking Closeouts,0.85,5.2,2,25.0,0.97,6.7,33.3,2,50.0,1,0.0,0.0,2.0,0.0,1,0.0,0,N/A
Catch & Shoot,1.71,4.4,0,57.1,1.71,6.5,57.1,0,N/A,7,57.1,N/A,0.0,N/A,0,N/A,0,N/A


In [30]:
butlerStats = PPP(ovr_player_data['butler'])
butlerStats.to_csv('!PPP Stats/butlerStats.csv')
butlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.89,29.8,8,38.5,0.78,27.3,32.9,59,33.9,11,27.3,1.08,35.4,48.7,19,47.4,20,50.0
PNR Screener,0.0,0.7,0,0.0,0.0,0.7,0.0,2,0.0,0,N/A,0.0,0.8,0.0,0,N/A,1,0.0
DHO Ball Handler,1.16,1.0,0,33.3,1.16,1.5,33.3,2,0.0,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.81,2.4,1,37.5,1.14,0.3,N/A,0,N/A,0,N/A,0.78,6.9,37.5,3,66.7,5,20.0
DBL Ball Handler,0.8,1.2,0,40.0,1.33,1.0,66.7,2,100.0,1,0.0,0.0,1.5,0.0,0,N/A,2,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.08,43.1,9,45.2,0.92,42.5,37.6,83,39.8,18,27.8,1.44,44.6,59.3,18,72.2,36,52.8
TRANSITION,1.46,6.4,2,73.7,1.46,6.9,71.4,14,71.4,0,N/A,1.45,5.3,80.0,3,66.7,2,100.0
Attacking Closeouts,0.77,3.4,4,37.5,0.83,4.6,42.9,6,50.0,1,0.0,0.0,0.8,0.0,1,0.0,0,N/A
Catch & Shoot,2.09,2.6,0,72.7,2.09,3.8,72.7,1,100.0,10,70.0,N/A,0.0,N/A,0,N/A,0,N/A


In [31]:
newAdebayoStats = PPP(new_player_data['adebayo'])
newAdebayoStats.to_csv(f'Adebayo/PPP Stats/{game + Opp}.csv')
newAdebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.29,30.4,1,16.7,0.4,25.0,20.0,5,20.0,0,N/A,0.0,66.7,0.0,0,N/A,1,0.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.33,13.0,0,66.7,1.33,15.0,66.7,3,66.7,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.64,21.2,0,75.0,1.64,24.4,75.0,4,75.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.0,8.7,0,50.0,1.0,10.0,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,8.7,0,0.0,0.0,10.0,0.0,2,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [32]:
newAdebayoSeriesStats = PPP(ovr_series_player_data['adebayo'])
newAdebayoSeriesStats.to_csv(f'Adebayo/PPP Stats/{series}/{Opp + series}.csv')
newAdebayoSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.0,1.5,0,50.0,1.0,1.9,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.91,21.0,4,42.9,0.96,22.7,42.1,19,42.1,0,N/A,0.68,14.8,50.0,1,100.0,1,0.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.05,16.8,1,47.6,0.92,12.3,46.2,13,46.2,0,N/A,1.21,32.9,50.0,2,100.0,6,33.3
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.83,29.1,7,46.7,0.98,29.8,46.4,28,46.4,0,N/A,0.25,26.7,50.0,1,100.0,1,0.0
TRANSITION,1.31,10.7,1,60.0,1.3,10.9,57.1,7,57.1,0,N/A,1.33,10.0,66.7,2,100.0,1,0.0
Attacking Closeouts,0.8,3.7,0,40.0,0.5,3.8,25.0,4,25.0,0,N/A,2.0,3.3,100.0,1,100.0,0,N/A
Catch & Shoot,2.0,0.7,0,100.0,2.0,0.9,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [33]:
adebayoStats = PPP(ovr_player_data['adebayo'])
adebayoStats.to_csv('!PPP Stats/adebayoStats.csv')
adebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.36,2.0,0,50.0,1.36,2.9,50.0,4,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.0,18.7,15,59.5,0.99,21.3,56.8,37,56.8,0,N/A,1.05,12.2,80.0,1,100.0,4,75.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.97,17.6,2,40.4,0.74,9.4,36.8,19,36.8,0,N/A,1.12,38.0,42.4,10,60.0,23,34.8
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.97,37.7,15,50.5,0.94,41.0,45.7,81,45.7,0,N/A,1.07,29.8,70.0,15,80.0,5,40.0
TRANSITION,1.39,9.0,2,65.2,1.31,7.6,66.7,12,66.7,0,N/A,1.52,12.6,63.6,5,100.0,6,33.3
Attacking Closeouts,0.86,2.1,0,42.9,0.67,2.6,33.3,6,33.3,0,N/A,2.0,1.1,100.0,1,100.0,0,N/A
Catch & Shoot,2.0,0.6,0,100.0,2.0,0.9,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [34]:
newLoveStats = PPP(new_player_data['love'])
newLoveStats.to_csv(f'Love/PPP Stats/{game + Opp}.csv')
newLoveStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,0.75,100.0,0,25.0,0.75,100.0,25.0,0,N/A,4,25.0,N/A,N/A,N/A,0,N/A,0,N/A


In [35]:
LoveSeriesStats = PPP(ovr_series_player_data['love'])
LoveSeriesStats.to_csv(f'Love/PPP Stats/{series}/{Opp + series}.csv')
LoveSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,4.5,0,0.0,0.0,4.5,0.0,0,N/A,1,0.0,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,4.5,0,0.0,0.0,4.5,0.0,0,N/A,1,0.0,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,3.0,4.5,0,100.0,3.0,4.5,100.0,0,N/A,1,100.0,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,2.27,4.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,2.27,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,1.4,29.3,0,50.0,1.4,29.3,50.0,1,100.0,5,40.0,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,1.25,54.5,0,41.7,1.25,54.5,41.7,0,N/A,12,41.7,N/A,N/A,N/A,0,N/A,0,N/A


In [36]:
loveStats = PPP(ovr_player_data['love'])
loveStats.to_csv('!PPP Stats/loveStats.csv')
loveStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.0,4.4,1,0.0,0.0,2.6,0.0,1,0.0,0,N/A,0.0,14.3,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,4.4,1,0.0,0.0,2.6,0.0,0,N/A,1,0.0,0.0,14.3,N/A,0,N/A,0,N/A
DHO Screener,0.0,2.2,0,0.0,0.0,2.6,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.5,17.8,0,25.0,0.29,18.4,14.3,7,14.3,0,N/A,2.0,14.3,100.0,1,100.0,0,N/A
TRANSITION,1.6,11.1,0,60.0,1.5,10.5,50.0,0,N/A,4,50.0,2.0,14.3,100.0,1,100.0,0,N/A
Attacking Closeouts,1.35,9.9,0,50.0,1.35,11.7,50.0,1,100.0,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.71,37.8,1,62.5,1.62,42.1,60.0,1,100.0,14,57.1,3.0,14.3,100.0,0,N/A,1,100.0


In [37]:
newLowryStats = PPP(new_player_data['lowry'])
newLowryStats.to_csv(f'Lowry/PPP Stats/{game + Opp}.csv')
newLowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.56,37.5,1,25.0,0.6,35.7,25.0,1,0.0,3,33.3,0.5,40.0,25.0,4,25.0,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.52,5.5,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,1.52,13.2,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.75,16.7,0,25.0,0.75,28.6,25.0,2,0.0,2,50.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.2,20.8,0,60.0,0.0,7.1,0.0,1,0.0,0,N/A,1.5,40.0,75.0,3,100.0,1,0.0
Attacking Closeouts,0.0,4.2,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,10.0,0.0,0,N/A,1,0.0
Catch & Shoot,1.0,12.5,0,33.3,1.0,21.4,33.3,0,N/A,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [38]:
LowrySeriesStats = PPP(ovr_series_player_data['lowry'])
LowrySeriesStats.to_csv(f'Lowry/PPP Stats/{series}/{Opp + series}.csv')
LowrySeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.28,37.2,2,50.0,1.41,34.8,50.0,4,25.0,8,62.5,1.16,39.8,50.0,11,45.5,3,66.7
PNR Screener,2.27,1.0,0,N/A,2.27,2.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.62,9.3,0,25.0,1.0,11.1,40.0,2,50.0,3,33.3,0.0,7.3,0.0,2,0.0,1,0.0
DHO Screener,2.16,2.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.16,5.7,100.0,0,N/A,1,100.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.96,12.1,2,50.0,1.0,15.6,50.0,4,50.0,2,50.0,0.87,8.4,50.0,1,100.0,1,0.0
TRANSITION,1.28,17.3,2,66.7,0.75,8.9,33.3,1,0.0,2,50.0,1.47,26.5,77.8,7,100.0,2,0.0
Attacking Closeouts,1.31,8.0,0,50.0,2.08,6.4,100.0,2,100.0,0,N/A,0.75,9.8,25.0,1,0.0,3,33.3
Catch & Shoot,0.67,10.5,0,22.2,0.75,17.8,25.0,0,N/A,8,25.0,0.0,2.4,0.0,1,0.0,0,N/A


In [39]:
lowryStats = PPP(ovr_player_data['lowry'])
lowryStats.to_csv('!PPP Stats/lowryStats.csv')
lowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.07,42.2,10,46.8,1.01,44.2,40.5,19,26.3,18,55.6,1.15,39.9,56.0,19,52.6,6,66.7
PNR Screener,2.27,0.5,0,N/A,2.27,0.9,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.69,7.1,0,30.8,1.0,8.9,44.4,4,75.0,5,20.0,0.0,4.8,0.0,2,0.0,2,0.0
DHO Screener,2.16,1.3,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.16,2.8,100.0,0,N/A,1,100.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.1,11.9,5,62.5,0.93,13.9,60.0,8,62.5,2,50.0,1.4,9.5,66.7,3,100.0,3,33.3
TRANSITION,1.53,17.8,3,71.4,1.02,7.8,33.3,3,0.0,3,66.7,1.69,30.0,81.8,15,93.3,7,57.1
Attacking Closeouts,1.24,7.0,1,54.5,1.7,5.8,80.0,5,80.0,0,N/A,0.86,8.4,33.3,1,0.0,5,40.0
Catch & Shoot,0.75,8.7,0,25.0,0.8,14.9,26.7,0,N/A,15,26.7,0.0,1.2,0.0,1,0.0,0,N/A


In [40]:
newMartinStats = PPP(new_player_data['martin'])
newMartinStats.to_csv(f'Martin/PPP Stats/{game + Opp}.csv')
newMartinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,2.0,9.1,0,100.0,2.0,12.5,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,9.1,0,0.0,0.0,12.5,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.5,18.2,0,50.0,0.0,12.5,0.0,0,N/A,1,0.0,3.0,33.3,100.0,0,N/A,1,100.0
TRANSITION,1.33,27.3,1,100.0,2.0,25.0,100.0,2,100.0,0,N/A,0.0,33.3,N/A,0,N/A,0,N/A
Attacking Closeouts,2.13,17.1,0,100.0,2.13,23.5,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,18.2,0,0.0,0.0,25.0,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [41]:
MartinSeriesStats = PPP(ovr_series_player_data['martin'])
MartinSeriesStats.to_csv(f'Martin/PPP Stats/{series}/{Opp + series}.csv')
MartinSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.5,7.4,0,25.0,0.5,10.0,25.0,3,33.3,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,3.7,0,0.0,0.0,5.0,0.0,1,0.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,5.6,0,0.0,0.0,2.5,0.0,1,0.0,0,N/A,0.0,14.3,0.0,1,0.0,1,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.5,18.5,1,22.2,0.22,22.5,12.5,6,16.7,2,0.0,3.0,7.1,100.0,0,N/A,1,100.0
TRANSITION,1.43,13.0,2,100.0,1.67,15.0,100.0,5,100.0,0,N/A,0.0,7.1,N/A,0,N/A,0,N/A
Attacking Closeouts,1.01,22.0,0,36.4,0.68,14.7,20.0,3,33.3,2,0.0,1.33,42.9,50.0,1,100.0,5,40.0
Catch & Shoot,1.5,18.5,0,50.0,1.5,25.0,50.0,0,N/A,10,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [42]:
martinStats = PPP(ovr_player_data['martin'])
martinStats.to_csv('!PPP Stats/martinStats.csv')
martinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.55,6.4,1,30.0,0.55,8.0,30.0,9,33.3,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.67,1.8,0,33.3,0.67,2.2,33.3,1,100.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.0,3.5,0,33.3,1.0,4.3,33.3,3,0.0,3,66.7,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.86,4.1,0,28.6,0.0,0.7,0.0,1,0.0,0,N/A,1.0,18.2,33.3,2,0.0,4,50.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.66,18.6,3,32.1,0.65,20.2,30.8,17,47.1,9,0.0,0.75,12.1,50.0,0,N/A,2,50.0
TRANSITION,1.22,13.4,5,76.5,1.27,13.7,73.3,14,78.6,1,0.0,1.0,12.1,100.0,2,100.0,0,N/A
Attacking Closeouts,1.42,18.6,0,60.0,1.36,16.5,61.9,15,73.3,6,33.3,1.56,27.3,55.6,1,100.0,8,50.0
Catch & Shoot,1.85,23.4,0,62.5,1.82,28.3,61.5,1,100.0,38,60.5,3.0,3.0,100.0,0,N/A,1,100.0


In [43]:
newRobinsonStats = PPP(new_player_data['robinson'])
newRobinsonStats.to_csv(f'Robinson/PPP Stats/{game + Opp}.csv')
newRobinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.0,27.3,0,33.3,0.0,14.3,0.0,1,0.0,0,N/A,1.5,50.0,50.0,1,0.0,1,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.0,18.2,0,50.0,N/A,0.0,N/A,0,N/A,0,N/A,1.0,50.0,50.0,2,50.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,9.1,0,0.0,0.0,14.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.27,8.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,2.27,22.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,9.1,1,N/A,0.0,14.3,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.0,27.3,0,33.3,1.0,42.9,33.3,0,N/A,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [44]:
newRobinsonSeriesStats = PPP(ovr_series_player_data['robinson'])
newRobinsonSeriesStats.to_csv(f'Robinson/PPP Stats/{series}/{Opp + series}.csv')
newRobinsonSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.75,8.7,1,33.3,0.0,6.2,0.0,1,0.0,0,N/A,1.5,14.3,50.0,1,0.0,1,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.18,25.8,1,50.0,1.2,15.6,50.0,0,N/A,4,50.0,1.16,49.1,50.0,4,50.0,2,50.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.67,6.5,0,66.7,1.67,9.4,66.7,2,50.0,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.39,6.3,0,50.0,2.0,3.1,100.0,1,100.0,0,N/A,1.06,13.4,0.0,1,0.0,0,N/A
Attacking Closeouts,1.61,20.3,1,85.7,1.29,17.0,75.0,2,100.0,2,50.0,2.06,27.7,100.0,3,100.0,0,N/A
Catch & Shoot,1.5,21.7,0,50.0,1.5,31.2,50.0,0,N/A,10,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [45]:
robinsonStats = PPP(ovr_player_data['robinson'])
robinsonStats.to_csv('!PPP Stats/robinsonStats.csv')
robinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.31,13.2,2,57.1,0.8,11.5,37.5,3,33.3,5,40.0,2.17,17.6,83.3,3,66.7,3,100.0
PNR Screener,0.0,0.8,0,0.0,0.0,1.1,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.01,26.2,3,44.4,1.0,21.8,38.9,5,40.0,13,38.5,1.02,37.5,55.6,5,60.0,4,50.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.78,7.4,1,37.5,0.78,10.3,37.5,5,40.0,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.9,7.3,1,42.9,1.0,4.6,50.0,2,100.0,2,0.0,0.82,14.4,33.3,3,33.3,0,N/A
Attacking Closeouts,1.42,15.1,2,73.3,1.34,8.6,66.7,2,100.0,4,50.0,1.47,32.0,77.8,8,87.5,1,0.0
Catch & Shoot,1.58,15.7,1,55.6,1.58,21.8,55.6,0,N/A,18,55.6,N/A,0.0,N/A,0,N/A,0,N/A


In [46]:
newStrusStats = PPP(new_player_data['strus'])
newStrusStats.to_csv(f'Strus/PPP Stats/{game + Opp}.csv')
newStrusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,13.3,0,0.0,0.0,7.7,0.0,0,N/A,1,0.0,0.0,50.0,0.0,0,N/A,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,2.0,13.3,0,100.0,2.0,7.7,100.0,1,100.0,0,N/A,2.0,50.0,100.0,1,100.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.0,13.3,0,50.0,1.0,15.4,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.67,20.0,1,50.0,0.67,23.1,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,13.3,0,0.0,0.0,15.4,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [47]:
newStrusSeriesStats = PPP(ovr_series_player_data['strus'])
newStrusSeriesStats.to_csv(f'Strus/PPP Stats/{series}/{Opp + series}.csv')
newStrusSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.89,12.7,0,28.6,0.0,9.1,0.0,3,0.0,1,0.0,1.8,21.6,66.7,1,100.0,2,50.0
PNR Screener,2.0,4.8,0,66.7,2.0,6.8,66.7,0,N/A,3,66.7,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.22,19.9,0,54.5,1.0,4.5,50.0,1,100.0,1,0.0,1.26,57.3,55.6,7,57.1,2,50.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.33,4.8,0,66.7,1.0,4.5,50.0,2,50.0,0,N/A,2.0,5.6,100.0,1,100.0,0,N/A
Attacking Closeouts,0.86,15.0,1,42.9,0.63,14.4,25.0,3,33.3,1,0.0,1.33,16.7,66.7,2,100.0,1,0.0
Catch & Shoot,0.3,32.3,0,10.0,0.3,45.5,10.0,0,N/A,20,10.0,N/A,0.0,N/A,0,N/A,0,N/A


In [48]:
strusStats = PPP(ovr_player_data['strus'])
strusStats.to_csv('!PPP Stats/strusStats.csv')
strusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.04,18.4,2,43.5,0.89,16.4,37.5,7,28.6,9,44.4,1.4,25.4,57.1,4,75.0,3,33.3
PNR Screener,2.0,2.1,0,66.7,2.0,2.7,66.7,0,N/A,3,66.7,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.88,18.6,0,33.3,0.8,9.1,30.0,1,100.0,9,22.2,0.93,52.3,35.7,11,36.4,3,33.3
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,2.1,0,0.0,0.0,2.7,0.0,1,0.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.87,8.3,0,90.0,1.84,8.9,87.5,7,85.7,1,100.0,2.0,6.5,100.0,2,100.0,0,N/A
Attacking Closeouts,0.99,11.5,2,50.0,0.82,11.1,37.5,6,50.0,2,0.0,1.5,12.9,75.0,3,100.0,1,0.0
Catch & Shoot,0.63,28.3,0,20.5,0.62,35.5,20.5,0,N/A,39,20.5,1.14,2.8,N/A,0,N/A,0,N/A


In [49]:
newVincentStats = PPP(new_player_data['vincent'])
newVincentStats.to_csv(f'Vincent/PPP Stats/{game + Opp}.csv')
newVincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.12,47.1,1,57.1,0.8,38.5,50.0,3,66.7,1,0.0,1.67,75.0,66.7,2,50.0,1,100.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.0,11.8,0,50.0,0.0,7.7,0.0,1,0.0,0,N/A,2.0,25.0,100.0,1,100.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,11.8,0,0.0,0.0,15.4,0.0,2,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.0,11.8,0,50.0,1.0,15.4,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,17.6,0,0.0,0.0,23.1,0.0,0,N/A,3,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [50]:
newVincentSeriesStats = PPP(ovr_series_player_data['vincent'])
newVincentSeriesStats.to_csv(f'Vincent/PPP Stats/{series}/{Opp + series}.csv')
newVincentSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.99,41.7,3,46.9,0.75,33.9,41.2,11,54.5,6,16.7,1.3,59.4,53.3,7,57.1,8,50.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.51,9.3,0,14.3,0.52,6.6,0.0,1,0.0,2,0.0,0.5,15.4,25.0,4,25.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,1.67,3.5,0,66.7,N/A,0.0,N/A,0,N/A,0,N/A,1.67,11.5,66.7,1,100.0,2,50.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,4.7,0,0.0,0.0,6.8,0.0,2,0.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.16,2.7,0,100.0,2.08,2.4,100.0,1,100.0,0,N/A,2.27,3.4,N/A,0,N/A,0,N/A
Attacking Closeouts,0.65,14.5,1,30.0,0.86,15.8,42.9,7,42.9,0,N/A,0.0,11.5,0.0,0,N/A,3,0.0
Catch & Shoot,1.42,22.4,0,47.4,1.42,32.2,47.4,0,N/A,19,47.4,N/A,0.0,N/A,0,N/A,0,N/A


In [51]:
vincentStats = PPP(ovr_player_data['vincent'])
vincentStats.to_csv('!PPP Stats/vincentStats.csv')
vincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.15,41.2,6,53.4,1.09,38.5,51.0,31,61.3,18,33.3,1.29,48.1,58.3,12,66.7,12,50.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.62,9.1,1,20.0,0.65,7.6,12.5,2,0.0,6,16.7,0.57,12.7,28.6,7,28.6,0,N/A
DHO Screener,3.0,0.5,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,1.8,100.0,0,N/A,1,100.0
DBL Ball Handler,1.35,4.5,0,50.0,1.0,2.1,33.3,2,0.0,1,100.0,1.53,10.7,60.0,2,100.0,3,33.3
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.32,8.0,3,9.1,0.36,9.8,9.1,7,14.3,4,0.0,0.0,3.6,N/A,0,N/A,0,N/A
TRANSITION,1.83,4.2,0,66.7,2.46,1.7,100.0,1,100.0,1,100.0,1.56,10.5,50.0,1,100.0,3,33.3
Attacking Closeouts,0.72,11.3,1,27.8,0.82,12.1,30.8,12,33.3,1,0.0,0.4,9.1,20.0,1,100.0,4,0.0
Catch & Shoot,1.58,18.4,0,52.8,1.58,25.5,52.8,0,N/A,36,52.8,N/A,0.0,N/A,0,N/A,0,N/A


In [ ]:
newHighsmithStats = PPP(new_player_data['highsmith'])
newHighsmithStats.to_csv(f'Highsmith/PPP Stats/{game + Opp}.csv')
newHighsmithStats

In [ ]:
HighsmithSeriesStats = PPP(ovr_series_player_data['highsmith'])
HighsmithSeriesStats.to_csv(f'Highsmith/PPP Stats/{series}/{Opp + series}.csv')
HighsmithSeriesStats

In [53]:
highsmithStats = PPP(ovr_player_data['highsmith'])
highsmithStats.to_csv('!PPP Stats/highsmithStats.csv')
highsmithStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,5.1,0,0.0,0.0,6.2,0.0,2,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,2.54,7.1,0,100.0,2.54,8.6,100.0,0,N/A,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,5.1,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,28.6,0.0,0,N/A,2,0.0
DHO Screener,0.0,5.1,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,28.6,0.0,0,N/A,2,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,2.0,5.1,0,100.0,2.0,6.2,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.0,2.6,0,100.0,2.0,3.1,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.6,25.6,2,100.0,1.6,31.2,100.0,8,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,5.1,0,0.0,0.0,6.2,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,23.1,0,66.7,2.0,28.1,66.7,0,N/A,9,66.7,N/A,0.0,N/A,0,N/A,0,N/A


In [54]:
newZellerStats = PPP(new_player_data['zeller'])
newZellerStats.to_csv(f'Zeller/PPP Stats/{game + Opp}.csv')
newZellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A


In [55]:
newZellerSeriesStats = PPP(ovr_series_player_data['zeller'])
newZellerSeriesStats.to_csv(f'Zeller/PPP Stats/{series}/{Opp + series}.csv')
newZellerSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,2.0,14.3,0,100.0,2.0,16.7,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,28.6,1,0.0,0.0,33.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [56]:
zellerStats = PPP(ovr_player_data['zeller'])
zellerStats.to_csv('!PPP Stats/zellerStats.csv')
zellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.0,24.0,1,60.0,1.33,16.7,66.7,3,66.7,0,N/A,0.67,42.9,50.0,1,100.0,1,0.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.0,4.0,0,100.0,2.0,5.6,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,3.5,0,N/A,0.0,4.9,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,16.0,1,0.0,0.0,22.2,0.0,3,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [67]:
newTeamStats = PPP(new_player_data['team'])
newTeamStats.to_csv(f'Team/PPP Stats/{game + Opp}.csv')
newTeamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.65,24.1,2,28.1,0.57,20.0,26.3,12,25.0,7,28.6,0.77,36.1,30.8,8,25.0,5,40.0
PNR Screener,0.29,5.0,1,16.7,0.4,4.8,20.0,5,20.0,0,N/A,0.0,5.6,0.0,0,N/A,1,0.0
DHO Ball Handler,1.2,5.9,0,57.1,0.93,4.1,33.3,2,50.0,1,0.0,1.5,11.1,75.0,4,75.0,0,N/A
DHO Screener,1.27,4.5,0,60.0,1.33,2.9,66.7,3,66.7,0,N/A,1.2,9.2,50.0,1,100.0,1,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.89,17.6,1,36.4,0.82,19.8,31.6,15,33.3,4,25.0,1.25,11.1,66.7,1,100.0,2,50.0
TRANSITION,1.36,10.5,1,66.7,1.35,8.5,62.5,8,62.5,0,N/A,1.36,16.3,75.0,3,100.0,1,0.0
Attacking Closeouts,0.78,9.0,2,33.3,0.93,10.2,42.9,7,42.9,0,N/A,0.0,5.6,0.0,1,0.0,1,0.0
Catch & Shoot,0.63,13.5,0,21.1,0.63,18.1,21.1,0,N/A,19,21.1,N/A,0.0,N/A,0,N/A,0,N/A


In [74]:
newOppStats = PPP(new_player_data['opp'])
newOppStats.to_csv(f'Opp/PPP Stats/{game + Opp}.csv')
newOppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.87,10.2,3,50.0,0.86,7.1,42.9,5,60.0,2,0.0,0.88,16.3,60.0,3,66.7,2,50.0
PNR Screener,0.0,2.0,1,0.0,0.0,1.0,0.0,1,0.0,0,N/A,0.0,4.1,0.0,1,0.0,0,N/A
DHO Ball Handler,0.2,3.3,1,0.0,0.0,3.1,0.0,3,0.0,0,N/A,0.53,3.8,N/A,0,N/A,0,N/A
DHO Screener,0.0,0.7,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,2.0,0.0,1,0.0,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.87,22.6,7,54.2,0.98,18.7,66.7,12,66.7,0,N/A,0.74,30.4,41.7,5,100.0,7,0.0
TRANSITION,0.98,20.8,1,42.3,1.13,18.0,46.7,11,54.5,4,25.0,0.78,26.3,36.4,6,50.0,5,20.0
Attacking Closeouts,1.26,9.2,1,77.8,1.24,9.8,83.3,6,83.3,0,N/A,1.29,7.9,66.7,1,100.0,2,50.0
Catch & Shoot,0.55,15.0,1,19.0,0.57,21.4,19.0,0,N/A,21,19.0,0.0,2.0,N/A,0,N/A,0,N/A


In [68]:
TeamSeriesStats = PPP(ovr_series_player_data['team'])
TeamSeriesStats.to_csv(f'Team/PPP Stats/{series}/{Opp + series}.csv')
TeamSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.94,20.7,8,39.7,0.71,17.5,31.1,51,29.4,23,34.8,1.29,28.1,51.9,29,48.3,23,56.5
PNR Screener,1.08,5.2,4,46.2,1.13,6.5,45.8,21,42.9,3,66.7,0.68,2.2,50.0,1,100.0,1,0.0
DHO Ball Handler,0.86,6.8,1,34.1,0.8,4.4,27.8,6,33.3,12,25.0,0.91,12.3,39.1,17,41.2,6,33.3
DHO Screener,1.0,4.7,1,42.9,0.87,3.1,42.9,14,42.9,0,N/A,1.1,8.4,42.9,4,75.0,10,30.0
DBL Ball Handler,1.5,0.9,0,66.7,2.0,0.4,100.0,2,100.0,0,N/A,1.25,2.0,50.0,1,100.0,3,33.3
DBL Screener,2.5,0.3,0,100.0,2.5,0.4,100.0,1,100.0,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.94,18.0,12,43.7,0.93,18.3,40.8,64,43.8,12,25.0,0.99,17.3,51.9,10,80.0,17,35.3
TRANSITION,1.35,8.7,6,67.4,1.35,8.1,67.9,26,69.2,2,50.0,1.36,10.0,66.7,12,83.3,3,0.0
Attacking Closeouts,1.0,10.6,6,43.1,0.93,10.6,40.5,25,48.0,12,25.0,1.14,10.7,47.6,9,77.8,12,25.0
Catch & Shoot,1.14,13.3,0,38.5,1.16,18.8,38.9,1,100.0,89,38.2,0.0,0.5,0.0,1,0.0,0,N/A


In [79]:
OppSeriesStats = PPP(ovr_series_player_data['opp'])
OppSeriesStats.to_csv(f'Opp/PPP Stats/{series}/{Opp + series}.csv')
OppSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.07,14.9,14,57.0,1.05,10.8,55.6,39,61.5,6,16.7,1.09,23.0,58.5,21,81.0,20,35.0
PNR Screener,1.13,3.6,3,57.1,1.48,3.2,69.2,8,87.5,5,40.0,0.6,4.3,37.5,6,50.0,2,0.0
DHO Ball Handler,1.02,5.5,5,44.8,0.9,5.0,42.1,13,38.5,6,50.0,1.19,6.5,50.0,6,50.0,4,50.0
DHO Screener,1.3,4.1,1,50.0,1.85,2.0,71.4,4,75.0,3,66.7,1.04,8.4,41.2,10,40.0,7,42.9
DBL Ball Handler,0.0,0.4,0,0.0,0.0,0.2,0.0,0,N/A,1,0.0,0.0,0.9,0.0,1,0.0,1,0.0
DBL Screener,0.67,0.4,0,33.3,0.0,0.4,0.0,1,0.0,1,0.0,2.0,0.4,100.0,1,100.0,0,N/A
ISOLATION,0.99,22.3,21,49.6,0.88,21.8,46.2,69,49.3,9,22.2,1.21,23.2,56.1,17,94.1,24,29.2
TRANSITION,1.16,14.1,6,49.3,1.16,11.9,51.3,32,59.4,7,14.3,1.16,18.4,47.2,14,64.3,22,36.4
Attacking Closeouts,1.35,6.2,1,62.9,1.16,6.1,56.5,19,57.9,4,50.0,1.71,6.3,75.0,5,100.0,7,57.1
Catch & Shoot,1.1,12.4,2,37.3,1.12,18.0,36.6,0,N/A,82,36.6,0.67,1.3,100.0,1,100.0,0,N/A


In [69]:
teamStats = PPP(ovr_player_data['team'])
teamStats.to_csv('!PPP Stats/teamStats.csv')
teamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.01,20.5,29,44.6,0.9,18.9,39.1,134,38.8,63,39.7,1.22,24.6,55.4,57,56.1,44,54.5
PNR Screener,1.02,4.9,17,54.2,1.06,5.5,52.9,44,54.5,7,42.9,0.85,3.4,62.5,2,100.0,6,50.0
DHO Ball Handler,0.83,6.1,5,33.0,0.9,5.0,32.7,17,35.3,38,31.6,0.74,8.8,33.3,25,36.0,11,27.3
DHO Screener,0.97,4.8,3,38.9,0.69,2.1,33.3,20,35.0,1,0.0,1.09,11.5,41.2,15,53.3,36,36.1
DBL Ball Handler,1.15,0.8,0,46.2,1.17,0.5,50.0,4,50.0,2,50.0,1.14,1.6,42.9,2,100.0,5,20.0
DBL Screener,2.33,0.2,0,100.0,2.33,0.3,100.0,2,100.0,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.96,23.5,36,44.6,0.85,24.6,38.6,211,42.2,38,18.4,1.28,20.8,62.7,37,78.4,46,50.0
TRANSITION,1.46,9.3,15,72.4,1.41,8.0,71.2,61,75.4,12,50.0,1.53,12.8,74.0,32,87.5,18,50.0
Attacking Closeouts,1.08,8.0,11,49.1,1.02,8.1,46.8,56,53.6,21,28.6,1.21,7.8,54.3,16,81.2,19,31.6
Catch & Shoot,1.42,11.4,2,48.7,1.41,15.7,48.4,5,100.0,181,47.0,1.8,0.8,66.7,1,0.0,2,100.0


In [80]:
oppStats = PPP(ovr_player_data['opp'])
oppStats.to_csv('!PPP Stats/oppStats.csv')
oppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.02,14.2,34,51.7,0.9,10.6,46.7,80,56.2,25,16.0,1.14,21.9,57.3,45,80.0,51,37.3
PNR Screener,1.3,4.6,9,61.5,1.52,4.3,68.9,27,85.2,18,44.4,0.9,5.2,45.0,8,50.0,12,41.7
DHO Ball Handler,1.13,3.1,6,48.8,1.04,2.8,44.8,16,43.8,13,46.2,1.29,3.8,57.1,8,62.5,6,50.0
DHO Screener,1.17,2.4,2,45.7,1.74,1.0,66.7,5,80.0,4,50.0,0.94,5.4,38.5,14,42.9,12,33.3
DBL Ball Handler,1.0,0.5,0,44.4,0.5,0.3,25.0,3,33.3,1,0.0,1.4,0.9,60.0,3,66.7,2,50.0
DBL Screener,1.57,0.4,0,71.4,1.4,0.4,60.0,3,66.7,2,50.0,2.0,0.4,100.0,2,100.0,0,N/A
ISOLATION,0.98,22.2,45,47.2,0.91,21.9,45.1,165,51.5,41,19.5,1.13,22.8,51.5,41,80.5,58,31.0
TRANSITION,1.13,14.8,24,50.5,1.12,12.5,51.4,85,60.0,24,20.8,1.14,19.7,49.4,36,75.0,49,30.6
Attacking Closeouts,1.1,9.8,12,48.9,1.01,9.7,47.1,67,52.2,20,30.0,1.3,9.9,52.1,13,84.6,35,40.0
Catch & Shoot,0.99,14.7,5,33.9,1.01,20.9,33.6,1,100.0,243,33.3,0.57,1.3,50.0,2,100.0,2,0.0
